In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt,fabs
data = pd.read_csv('kc_house_data.csv')
print data[0]

KeyError: 0

In [3]:
def get_numpy_data(data_sframe, features, output):
    data = pd.DataFrame()
    n = len(data_sframe)
    data['ones'] = np.array([1]*n)
    data[features] = data_sframe[features]
    data['output'] = data_sframe[output] 
    features = ['ones']+features
    return (data[features],data['output'])

# A function to predict the output

In [4]:
def predict_output(feature_matrix,weight):
    return feature_matrix.dot(weight)

# A function to normalize the features

In [5]:
def normalize_features(features):
    x = list(features)
    norms = []
    for feature in x:
        if feature == 'ones':
            features[feature] = features[feature]
            norms.append(1)
        else:
            temp = sqrt((features[feature]*features[feature]).sum())
            features[feature] = features[feature]/temp
            norms.append(temp)
    return features,norms
            
                
    

# A simple model with 2 features

In [6]:
features = ['sqft_living','bedrooms']
input_matrix,output = get_numpy_data(data,features,'price')

In [7]:
print input_matrix[0]
print output.head(2)

KeyError: 0

In [60]:
normalize_input_matrix,norms = normalize_features(input_matrix)

In [61]:
print normalize_input_matrix.head(2)

   ones  sqft_living  bedrooms
0     1     0.003530  0.005836
1     1     0.007689  0.005836


In [62]:
print norms

[1, 334257.2641230105, 514.0758698869263]


In [71]:
W0 = np.array([1,4,1])
prediction = predict_output(normalize_input_matrix,W0)

n = len(W0)
ro = [0]*n
i = 0
ro[0] = normalize_input_matrix['ones'].dot(output - prediction + W0[0]*normalize_input_matrix['ones'])
ro[1] = normalize_input_matrix['sqft_living'].dot(output - prediction + W0[1]*normalize_input_matrix['sqft_living'])
ro[2] = normalize_input_matrix['bedrooms'].dot(output - prediction + W0[2]*normalize_input_matrix['bedrooms'])
print ro

[11672924328.338215, 87939337.252184689, 80966557.911839828]


## FOR W[1] TO BE NON ZERO  -175878674.50436938<L1<175878674.50436938
## FOR W[2] TO BE ZERO -80966557.911839828<L1<80966557.911839828

# Finally -80966557.911839828<L1<80966557.911839828

In [110]:
def lasso_coordinate_descent_step(i,feature_matrix,output,weights,l1_penalty):
    features = list(feature_matrix)
    num = {'ones':0,'sqft_living':1,'bedrooms':2}
    W = np.array([weights[feature] for feature in features])
    predicton = predict_output(feature_matrix,W)
    ro = feature_matrix[i].dot(output - prediction + feature_matrix[i]*W[num[i]])
    if i == 'ones' :
        w = ro
    elif ro < -(l1_penalty/2):
        w = ro + l1_penalty/2
    elif ro > l1_penalty/2 :
        w = ro - l1_penalty/2
    else:
        w = 0
    return w
            

In [113]:
def lasso_cyclical_coordinate_descent(feature_matrix,output,initial_weights,l1_penalty,tolerance):
    features = list(feature_matrix)
    W = {}
    i = 0
    for feature in ['ones','sqft_living','bedrooms']:
        W[feature] = initial_weights[i]
        i += 1
    while True:
        for feature in ['ones','sqft_living','bedrooms'] :
            old_weight = np.array([W[feature] for feature in features])
            W[feature] = lasso_coordinate_descent_step(feature,feature_matrix,output,W,l1_penalty)
            new_weight = np.array([W[feature] for feature in features])
            diff = [fabs(x) for x in old_weight - new_weight]
            if max(diff) < tolerance :
                return W
        print W
            

In [114]:

initial_weights = np.array([0]*3)
print initial_weights
l1_penalty = 1e7
tolerance = 1.0
feature_matrix = normalize_input_matrix
output = output
lasso_cyclical_coordinate_descent(feature_matrix,output,initial_weights,l1_penalty,tolerance)

[0 0 0]
{'sqft_living': 0, 'bedrooms': 227899670.73551965, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 455799341.47103924, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 683699012.20655918, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 911598682.94207871, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 1139498353.6775985, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 1367398024.4131169, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 1595297695.1486366, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 1823197365.8841565, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 2051097036.6196766, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 2278996707.3551979, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 2506896378.0907207, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 2734796048.8262396, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 2962695719.5617595, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 3190595390.2972827, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 3418495061.0328031, 'ones': 0}
{'sqft_living': 0, 'bedrooms': 3646394731.7683244, 'ones': 0}


KeyboardInterrupt: 